In [4]:
!pip install surprisal

In [2]:
import numpy as np
import pandas as pd
import re
import codecs
import ast
import openai
import json

### get surprisal data from GPT-3 (pre-trained)

In [66]:
!python get_surprisal_data.py

finished computing surprisals of 16 sentences


In [97]:
df = pd.read_csv('raw_data.csv')
df

,Unnamed: 0,tokens,surprisals,cond
0,0,"['bytes:\\xe6', 'bytes:\\x9b', 'bytes:\\xb8', ...",[ 6.6900610e+00 3.7060520e+00 2.8116540e+00 ...,island
1,1,"['bytes:\\xe7', 'bytes:\\x99', 'bytes:\\xba', ...",[ 6.1963797e+00 3.6777165e+00 2.4309633e+00 ...,island
2,2,"['bytes:\\xe8', 'bytes:\\xb3', 'bytes:\\xbc', ...",[ 6.75180630e+00 4.79560570e+00 1.80590090e+...,island
3,3,"['bytes:\\xe6', 'bytes:\\xbc', 'bytes:\\x94', ...",[ 6.69303660e+00 5.31987900e+00 1.33068450e+...,island
4,4,"['bytes:\\xe9\\x96', 'bytes:\\x8b', 'bytes:\\x...",[1.1540866e+01 9.3011490e-01 1.7977983e+00 6.2...,island
5,5,"['デ', 'ザ', 'イ', 'ン', 'し', 'た', 'ド', 'レ', 'ス', ...",[ 1.2742895e+01 2.4567206e+00 2.5391560e-02 ...,island
6,6,"['作', 'っ', 'た', 'bytes:\\xe6\\x97', 'bytes:\\x...",[ 1.0153272e+01 5.5638266e+00 6.2275780e-01 ...,island
7,7,"['bytes:\\xe5\\x8f', 'bytes:\\x97', 'bytes:\\x...",[8.43501700e+00 3.05482130e+00 2.86327270e+00 ...,island
8,8,"['bytes:\\xe6', 'bytes:\\x9b', 'bytes:\\xb8', ...",[ 6.6882844e+00 3.7083158e+00 2.8085725e+00 ...,non-island
9,9,"['bytes:\\xe7', 'bytes:\\x99', 'bytes:\\xba', ...",[ 6.1981072e+00 3.6763346e+00 2.4333370e+00 ...,non-island


### Convert bytes to characters

In [99]:
def decode_tokens(tokens, surprisals):
    tokens_new = []
    surprisals_new = []
    current_ix = -1
    ix_to_skip = []

    for to, su in zip(tokens, surprisals):
        current_ix += 1
        if len(to.split('\\'))==1:
            tokens_new.append(to)
            surprisals_new.append(su)
        elif len(to.split('\\'))==2:
            if len(tokens[current_ix+1].split('\\'))==2 and len(tokens[current_ix+2].split('\\'))==2 and current_ix not in ix_to_skip:
                byte_str = '\\' + '\\'.join([to.split('\\')[1], tokens[current_ix+1].split('\\')[1], tokens[current_ix+2].split('\\')[1]])
                converted = codecs.escape_decode(byte_str)[0].decode('utf8')
                tokens_new.append(converted)
                mean_su = (su + surprisals[current_ix+1] + surprisals[current_ix+2]) / 3
                surprisals_new.append(mean_su)
                ix_to_skip.extend([current_ix+1, current_ix+2])
            else:
                continue
        elif len(to.split('\\'))==3:
            byte_str = '\\' + '\\'.join([to.split('\\')[1], to.split('\\')[2], tokens[current_ix+1].split('\\')[1]])
            converted = codecs.escape_decode(byte_str)[0].decode('utf8')
            tokens_new.append(converted)
            mean_su = (su + surprisals[current_ix+1]) / 2
            surprisals_new.append(mean_su)
            ix_to_skip.extend([current_ix+1])
        elif len(to.split('\\'))==4:
            byte_str = '\\' + '\\'.join([to.split('\\')[1], to.split('\\')[2], to.split('\\')[3]])
            converted = codecs.escape_decode(byte_str)[0].decode('utf8')
            tokens_new.append(converted)
            surprisals_new.append(su)

    return tokens_new, surprisals_new

In [99]:
dat_new = []

for _, row in df.iterrows():
    surps = [x.replace('[','').replace(']','') for x in row['surprisals'].split(' ')]
    surps = [float(x.strip()) for x in surps if x != '']
    tokens_new, surprisals_new = decode_tokens(ast.literal_eval(row['tokens']), surps)
    updated_dict = {'tokens':tokens_new, 'surprisals':surprisals_new, 'island':[row['cond']]*len(tokens_new)}
    dat_new.append(updated_dict)

df_new = pd.DataFrame(dat_new)
df_new = df_new.explode(['tokens', 'surprisals', 'island'])

df_new.head(10)

,tokens,surprisals,island
0,書,4.402589,island
0,い,3.098066,island
0,た,1.065531,island
0,エ,8.526271,island
0,ッ,3.613167,island
0,セ,0.027428,island
0,ー,2.039104,island
0,が,7.167349,island
0,権,3.228217,island
0,威,0.426591,island


In [49]:
df_new.to_csv('result_GPT3_pretrained.csv')

In [50]:
result_edited = pd.read_csv('result_GPT3_pretrained.csv')

critical = result_edited[result_edited['critical_test']==1]
critical.groupby('island')['surprisals'].mean()

island
island        2.422033
non-island    2.823154
Name: surprisals, dtype: float64

### get acceptability data from GPT-3 (pre-trained)

In [22]:
from getpass import getpass
secret = getpass('Enter the secret value: ')

Enter the secret value: ········


In [23]:
openai.api_key = secret

In [5]:
with open('prompt.txt', 'r') as f:
    test_prompt = f.read()
test_prompt

'文章: 人気を博した舞台での踊りだけ方を努力家のダンサーは上達させた。\n評価: この文章は文法的ではありません。\n###\n文章: 熱血な教師が落ちこぼれの生徒を名門大学に合格させた。\n評価: この文章は文法的です。\n###\n文章: 保育士が正直な子どもが親が嘘をついたと言ったと思っている。\n評価: この文章は文法的ですが、複雑です。\n###\n文章: その農家の必死に育てた農作物の畑の腐られ方は大変気の毒だ。\n評価: この文章は文法的ではありません。\n###\n文章: 経験の浅い一年生が新人戦で三人走った。\n評価: この文章は文法的ですが、複雑です。\n###\n文章: '

In [20]:
test_stimuli = pd.read_csv('test_stimuli_new.csv')
test_stimuli

,island,sentence
0,0,書いたエッセーが権威ある文学賞を受賞したので女性議員は感謝を述べた。
1,0,発表した論文が世界中の学者から注目されたので教授は多忙を極めた。
2,0,購入した絵画がピカソの作品だったので美術館長は展示会を計画した。
3,0,演じた芝居が連日チケットが完売したので役者は海外に招待された。
4,0,開発した新製品が今朝社外に発表されたので若手社員は誇りに思った。
5,0,デザインしたドレスが大勢の記者に取材されたので女優は自信を付けた。
6,0,作った日本酒が先月ガイドブックで特集されたので外国人は周りに認められた。
7,0,受けた難病の診断が今朝社内で公表されたので重役は引き継ぎを始めた。
8,1,書いたエッセーが権威ある文学賞を受賞した女性議員は感謝を述べた。
9,1,発表した論文が世界中の学者から注目された教授は多忙を極めた。


In [34]:
def GPT3_model(prompt):
    response = openai.Completion.create(
    engine="davinci",
    prompt=prompt,
    max_tokens=50,
    temperature=0.7,
    top_p=1,
    n=1,
    stream=None)
    return response

In [44]:
judgments = []

for sent in test_stimuli['sentence'].tolist():
    prompt = test_prompt + sent + '\n' + '評価: '
    response = GPT3_model(prompt)
    judgment = response['choices'][0]['text'].split('\n')[0]
    judgments.append(judgment)

test_stimuli['judgment'] = judgments
test_stimuli.head()

### finetuning

In [53]:
finetuning_data = pd.read_csv('finetuning_stimuli_gpt3.csv')

In [55]:
finetuning_data_dict = []
for _, row in finetuning_data.iterrows():
    finetuning_data_dict.append({'prompt':row['prompt'], 'completion':row['completion']})
finetuning_data_dict

In [61]:
with open("finetuning_data.json", 'w') as f:
    for item in finetuning_data_dict:
        #f.write(json.dumps(item) + "\n")
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

![alt text](finetuning.png "Title")

### get surprisal data from GPT-3 (finetuned)

In [78]:
!python get_surprisal_data.py

finished computing surprisals of 16 sentences


In [98]:
df = pd.read_csv('raw_data_ft.csv')

In [101]:
dat_new = []

for i, row in df.iterrows():
    surps = [x.replace('[','').replace(']','') for x in row['surprisals'].split(' ')]
    surps = [float(x.strip()) for x in surps if x != '']
    tokens_new, surprisals_new = decode_tokens(ast.literal_eval(row['tokens']), surps)
    updated_dict = {'tokens':tokens_new, 'surprisals':surprisals_new, 'island':[row['cond']]*len(tokens_new)}
    dat_new.append(updated_dict)

df_new = pd.DataFrame(dat_new)
df_new = df_new.explode(['tokens', 'surprisals', 'island'])

In [102]:
df_new.head(10)

,tokens,surprisals,island
0,書,4.666651,non-island
0,い,2.363109,non-island
0,た,0.623235,non-island
0,エ,6.475675,non-island
0,ッ,1.872293,non-island
0,セ,0.210575,non-island
0,ー,3.064842,non-island
0,が,1.02555,non-island
0,権,3.237404,non-island
0,威,0.131714,non-island


In [103]:
df_new.to_csv('result_GPT3_finetuned.csv')

In [104]:
result_edited = pd.read_csv('result_GPT3_finetuned.csv')

critical = result_edited[result_edited['critical']==1]
critical.groupby('island')['surprisals'].mean()

island
island        1.613939
non-island    1.939812
Name: surprisals, dtype: float64

### get acceptability data from GPT-3 (finetuned)

In [3]:
test_stimuli = pd.read_csv('test_stimuli.csv')

In [6]:
judgments_ft = []

for sent in test_stimuli['sentence'].tolist():
    prompt = test_prompt + sent + '\n' + '評価: '
    response = openai.Completion.create(
    model='curie:ft-personal-2022-11-21-16-39-49',
    prompt=prompt,
    max_tokens=50)
    judgment = response['choices'][0]['text'].split('\n')[0]
    judgments_ft.append(judgment)

test_stimuli['judgment_ft'] = judgments_ft
test_stimuli.tail()

,Unnamed: 0,island,sentence,judgment,judgment_en,judgment_ft
11,11,1,演じた芝居が連日チケットが完売した役者は海外に招待された。,この文章は文法的です。,This sentence is grammatical.,ルビがきついです。この文章は文法的ですが、複雑です。
12,12,1,開発した新製品が今朝社外に発表された若手社員は誇りに思った。,この文章は文法的です。,This sentence is grammatical.,主語は文法的ですが、複雑です。この文章は文法的ですが、複雑です。この文
13,13,1,デザインしたドレスが大勢の記者に取材された女優は自信を付けた。,この文章は文法的です。,This sentence is grammatical.,人物が主語を修飾しています。この文章は文法的ですが、複雑です。この文章は
14,14,1,作った日本酒が先月ガイドブックで特集された外国人は周りに認められた。,この文章は文法的です。,This sentence is grammatical.,主語は特殊です。この文章は文法的ではありません。この文章は複雑です。
15,15,1,受けた難病の診断が今朝社内で公表された重役は引き継ぎを始めた。,この文章は文法的です。,This sentence is grammatical.,主語は文法的ですが、複雑です。
